合同细节

In [ ]:
from ib_async import *
util.startLoop()

import logging
# util.logToConsole(logging.DEBUG)

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=11)

假设我们想要查找 AMD 股票的合同详情。
让我们创建一个股票对象，并请求其详情：

In [ ]:
amd = Stock('AMD')

cds = ib.reqContractDetails(amd)

len(cds)

我们得到了一个很长的合同细节清单。让我们打印第一个：

In [ ]:
cds[0]

合同本身位于合同细节的“合同”属性。让我们列出合同并查看第一个合同：

In [ ]:
contracts = [cd.contract for cd in cds]

contracts[0]

为了更好地区分所有合同之间的差异，将其转换为DataFrame是很方便的。有一个实用函数可以做到这一点：

In [ ]:
util.df(contracts)

从这可以看出，AMD在不同交易所以不同货币进行交易。假设我们想要在SMART交易所以美元进行交易。那么AMD合同将进行调整，从而变得独一无二：

In [ ]:
amd = Stock('AMD', 'SMART', 'USD')

assert len(ib.reqContractDetails(amd)) == 1

让我们尝试同样的方法来检验英特尔：

In [ ]:
intc = Stock('INTC', 'SMART', 'USD')

assert len(ib.reqContractDetails(intc)) == 1

让我们试试一个不存在的合同。

In [ ]:
xxx = Stock('XXX', 'SMART', 'USD')

assert len(ib.reqContractDetails(xxx)) == 0

或者外汇合同

In [ ]:
eurusd = Forex('EURUSD')

assert len(ib.reqContractDetails(eurusd)) == 1

使用“qualifyContracts”方法，回传的合同详情请求中附加的信息用于填写原始合同。

让我们使用“amd”进行此操作，并在此之前和之后进行比较：

In [ ]:
amd

In [ ]:
ib.qualifyContracts(amd)
amd

提示：在打印合同时，输出可以复制粘贴，这将是有效的Python代码。

返回的“conId”本身可以用于唯一地指定一个合同：

In [ ]:
contract_4391 = Contract(conId=4391)

ib.qualifyContracts(contract_4391)

assert contract_4391 == amd

一大堆合同可以同时被认定资格。返回所有成功的合同的列表：

In [ ]:
qualContracts = ib.qualifyContracts(amd, intc, xxx, eurusd)

assert intc in qualContracts
assert xxx not in qualContracts

还有一个API功能可以请求符合模式的股票（仅限股票）。

In [ ]:
matches = ib.reqMatchingSymbols('intc')
matchContracts = [m.contract for m in matches]

matches

In [ ]:
assert intc in matchContracts

In [ ]:
ib.disconnect()

In [2]:
from ib_async import *
util.startLoop()

import logging
# util.logToConsole(logging.DEBUG)

# 连接到 TWS 或者 IB Gateway
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

account_summary = ib.reqAccountSummary()
print(account_summary)

# 定义你想要查询的股票
stock = Stock('AAPL', 'SMART', 'USD')


# 请求基本面数据（例如财报数据）
report_type = 'CalendarReport'  # 使用 IB API 的 'CalendarReport' 类型获取财报日期
fundamental_data = ib.reqFundamentalData(stock, report_type)

# 输出财报数据
print(fundamental_data)


# 断开连接
ib.disconnect()

Error 321, reqId 2147483647: Error validating request.-'cB' : cause - The API interface is currently in Read-Only mode.
Error 321, reqId -1: Error validating request.-'b_' : cause - The API interface is currently in Read-Only mode.
open orders request timed out
completed orders request timed out


None


Error 430, reqId 4: The fundamentals data for the security specified is not available.Not allowed, contract: Stock(symbol='AAPL', exchange='SMART', currency='USD')


[]


In [ ]:
import ib_async
from ib_fundamental import CompanyFinancials

# ib_async.util.startLoop() # if you are in a notebook

# connect to TWS API for ex on localhost:7497
ib = ib_async.IB().connect('localhost',7497)

# create your company financials instance
aapl = CompanyFinancials(ib=ib,symbol="AAPL")

# or specify exchange and currency
# aapl = CompanyFinancials(ib=ib,symbol="AAPL",exchange='SMART',currency='USD)

# get company info
aapl.company_information

# Annual income statement, while aapl.income_quarter will pull the quarterly report
aapl.income_annual

# get earnings per share, appl.eps_ttm will pull trailing twelve months eps
aapl.eps_ttm

# get data in json format

from ib_fundamental.utils import to_json

# CompanyFinancials.data property contains all data in dataclass format
to_json(aapl.data.eps_ttm)
'[{"as_of_date": "2024-03-31T00:00:00", "report_type": "TTM", "period": "12M", "eps": 6.46}, {"as_of_date": "2023-12-31T00:00:00", "report_type": "TTM", "period": "12M", "eps": 6.46}, ...'

# and much more